In [1]:
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
import util

In [3]:
XFILE = 'data/imu_proc01-Feb-2018.csv'
YFILE = 'data/vicon_proc01-Feb-2018.csv'
# X_NOV = 'data/imu_proc09-Nov-2017.csv'
# Y_NOV = 'data/vicon-9-nov.csv'

In [4]:
X = np.genfromtxt(XFILE, delimiter=',')
X = X[:, 10:]  # wrist only
print('Input size:', X.shape)
y = np.genfromtxt(YFILE, delimiter=',')
print('Output size:', y.shape)

Input size: (4731, 10)
Output size: (4731, 5)


In [5]:
X_train, X_test, y_train, y_test = util.test_split(X, y, 2000, 0.1)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
mlp = MLPRegressor(500, 'relu', max_iter=500, alpha=1)
svr = SVR(C=10, gamma=0.1, epsilon=1)
models = [svr, mlp]
names = ['SVM', 'MLP']

In [11]:
def test_score(model, X_train, X_test, y_train, y_test):
    # Obtain test accuracy
    y_pred = np.zeros_like(y_test)
    for yd in range(y_test.shape[1]):
        model.fit(X_train, y_train[:, yd])
        y_pred[:, yd] = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    print('Test:')
    print('r2: ', '\t'.join(map(str, r2)))
    print('rmse:', '\t'.join(map(str, rmse)))
    # np.savetxt('data/' + names[i] + '-true-' + YFILE.replace('data/', ''), y_test, '%.5f', delimiter=',')
    # np.savetxt('data/' + names[i] + '-pred-' + YFILE.replace('data/', ''), y_pred, '%.5f', delimiter=',')

In [12]:
print('=== SVM ===')
test_score(svr, X_train, X_test, y_train, y_test)

===SVR===
Test:
r2:  0.70936359578	0.74264176347	0.304301615742	0.827354523013	-0.386012086995
rmse: 6.56878969804	5.64436480939	7.11736176947	5.63737121646	5.29891334967
